<a href="https://colab.research.google.com/github/x110/MyFastaiNotebooks/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install git+https://github.com/fastai/fastai.git

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
from fastai.text import *

In [19]:
from spacy.symbols import ORTH
!python -m spacy download en

my_tok=spacy.load("en")
my_tok.tokenizer.add_special_case('<SUMM>',[{ORTH:'<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>',[{ORTH:'<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>',[{ORTH:'<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />',[{ORTH:'<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



## Get your data ready for modeling

In [0]:
PATH='drive/My Drive/myfastai/data/arxiv1/'

In [0]:
FILES = dict(train='trn', valid='val')

In [24]:
#<2.44
data = TextLMDataBunch.from_folder(f'{PATH}all/',**FILES,tokenizer=my_spacy_tok, min_freq=10)

AttributeError: ignored

Since this step can be a bit time-consuming, it's best to save the result with:

In [0]:
data.save()

This will create a 'tmp' directory where all the computed stuff will be stored. You can then reload those results with:

In [0]:
data = TextLMDataBunch.load(path)

## Fine-tuning a language model

We can use the data object we created earlier to fine-tune a pretrained language model. fast.ai has an English model available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.



In [0]:
learn = language_model_learner(data, pretrained_model=URLs.WT103_1, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

TextLMDataBunch;

Train: LabelList
y: LMLabel (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all
x: LMTextList (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all;

Valid: LabelList
y: LMLabel (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all
x: LMTextList (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all;

Test: None

In [0]:
learn.predict("This is a review about", n_words=10)

In [0]:
learn.save_encoder('ft_enc')